In [1]:
spotify_sdf=spark.read.parquet('gs://my-bucket-mpat/cleaned/Final_clean_data2.parquet')

24/04/20 00:07:34 WARN YarnScheduler: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/miniconda3/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
spotify_sdf.printSchema()

In [ ]:
spotify_sdf.count()

In [ ]:
train=.75
test=.25

In [ ]:
train, test = spotify_sdf.randomSplit([train, test], seed=42)

In [ ]:
print(train.count())

In [ ]:
print(test.count())

In [ ]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import VectorAssembler, StringIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
features = ['danceability', 'instrumentalness', 'liveness', 'energy', 'valence', 
            'key', 'speechiness', 'time_signature', 'tempo', 'acousticness']
label_col = 'playlist_uris'

# Select relevant columns from the DataFrame
data = spotify_sdf.select(features + [label_col])

# Assemble features into a single feature vector column
assembler = VectorAssembler(inputCols=features, outputCol="features")
feature_vector_df = assembler.transform(data)

# Convert categorical label column into numerical label column using StringIndexer
indexer = StringIndexer(inputCol=label_col, outputCol="label")
indexed_feature_vector_df = indexer.fit(feature_vector_df).transform(feature_vector_df)

# Split the data into training and testing sets (80% training, 20% testing)
(train_data, test_data) = indexed_feature_vector_df.randomSplit([0.8, 0.2], seed=123)

# Define and train a RandomForestClassifier
rf = RandomForestClassifier(featuresCol="features", labelCol="label")
rf_model = rf.fit(train_data)

# Make predictions on the test data
predictions = rf_model.transform(test_data)

# Evaluate the model using MulticlassClassificationEvaluator (e.g., accuracy)
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Accuracy:", accuracy)

# Stop the Spark session


In [ ]:
# Stop the existing SparkContext (if it exists)
sc.stop()
